# Hexagonal diagram creator

This project is a [Jupyter](https://en.wikipedia.org/wiki/Project_Jupyter) notebook that generates a hexagonal diagram using the information given as input: see content in CATEGORIES and HEXAGONS from the very next cell.

If you are not familiar with Jupyter notebook: it is as easy to use as scrolling down this page and reading both the Python code in successive cells and the partial result after each cell (if any). The entire notebook will be automatically run online sequentially for you from start to finish, all along this webpage. Scroll to the bottom to see the resulting diagram image.

You can get the source code above from this github repo, and run it in your own Jupyter notebook, which you can modify for your personal use.

Author: Ismael Aguilera.

In [1]:
# This cell contains the input that will describe the hexagons that has to be generated.

# Each category listed below each category will be assigned a specific filling color

CATEGORIES = [ "gp", "db", "web", "cd", "py", "data" ]

# This hexagon table defition contains a rectangular table with a topic, that will be printed 
# in the center of each cell, and a category (from list above) that will be used for each
# hexagon filling color.

HEXAGONS = [
    [
        {
            "topic": "Math|problem|solving",
            "category": "gp"
        },
        {
            "topic": "C++",
            "category": "gp"
        },
        {
            "topic": "C#",
            "category": "gp"
        },
        {
            "topic": "Java",
            "category": "gp"
        },
        {
            "topic": "AWS|DynamoDB",
            "category": "db"
        },
        {
            "topic": "AWS Aurora|& Postgres|+PostGIS",
            "category": "db"
        },
        {
            "topic": "MongoDB",
            "category": "db"
        }
    ],
    [
        {
            "topic": "GPU|computing|(CUDA)",
            "category": "gp"
        },
        {
            "topic": "Parallelism|& concurrency",
            "category": "gp"
        },
        {
            "topic": "AWS EC2|computing|(docker)",
            "category": "gp"
        },
        {
            "topic": "AWS Lambda",
            "category": "gp"
        },
        {
            "topic": "AWS|event-driven|architectures",
            "category": "gp"
        },
        {
            "topic": "AWS|Cloudfront+S3|& Amplify",
            "category": "web"
        },
        {
            "topic": "Auth|integration",
            "category": "web"
        }
    ],
    [
        {
            "topic": "Computational|geometry",
            "category": "gp"
        },
        {
            "topic": "GIS data|(Big data)",
            "category": "data"
        },
        {
            "topic": "AWS S3",
            "category": "data"
        },
        {
            "topic": "Python|+Fastapi|+Jupyter",
            "category": "py"
        },
        {
            "topic": "OpenAPI",
            "category": "py"
        },
        {
            "topic": "Node|+Express.js",
            "category": "py"
        },
        {
            "topic": "Angular",
            "category": "web"
        },
        None
    ],
    [
        {
            "topic": "3D|processing",
            "category": "data"
        },
        {
            "topic": "Image|processing|(OpenCV)",
            "category": "data"
        },
        {
            "topic": "AWS|CloudWatch|(monitoring)",
            "category": "cd"
        },
        {
            "topic": "Unit &|integration|testing",
            "category": "cd"
        },
        {
            "topic": "CI/CD",
            "category": "cd"
        },
        {
            "topic": "End-to-end|testing",
            "category": "cd"
        },
        {
            "topic": "Material|Bootstrap|CSS",
            "category": "web"
        }
    ]
]

In [2]:
import colorsys

class PaleColorPalette:
    """This class generates a palette of pale colors"""
    
    @classmethod
    def __init__(self, numColors):
        hueOffset = 1.0 / numColors
        saturation = 0.1
        brightness = 0.98
        
        self.PALE_COLORS = []

        for i in range(numColors):
            r, g, b = colorsys.hsv_to_rgb(i * hueOffset, saturation, brightness)

            color = 255 * r, 255 * g, 255 * b

            self.PALE_COLORS.append(color)


In [3]:
import sys, math
    
class HexSVG:
    """HexSVG class definition: This class creates an SVG of hexagons"""
    
    def __init__(self):
        """Initialization, it contains a set of magic numbers"""
        self.hexSize = 60
        self.hexLineSize = 57
        self.offsetX = self.hexSize * (1.5)
        self.offsetY = self.hexSize * 2 * math.sin(math.pi / 3)
        self.evenOffsetX = 0
        self.evenOffsetY = self.hexSize * math.sin(math.pi / 3)

        self.stringHeight = 20
    
        self.sb = ""
        
        self.minx = sys.float_info.max
        self.miny = sys.float_info.max
        self.maxx = -sys.float_info.max
        self.maxy = -sys.float_info.max
    
    def addHex(self, i, j, message, fillColor):
        """Adds an hexagon at position i, j with given message an filling color"""
        
        offx = (1 + i) * self.offsetX + (0 if j % 2 == 0 else self.evenOffsetX)
        offy = (1 + j) * self.offsetY + (0 if i % 2 == 0 else self.evenOffsetY)

        self.sb += "<polygon points=\""

        for s in range(6):
            linex1, liney1, linex2, liney2 = (\
                    offx + self.hexLineSize * math.cos(math.pi * s / 3), \
                    offy + self.hexLineSize * math.sin(math.pi * s / 3), \
                    offx + self.hexLineSize * math.cos(math.pi * (s + 1) / 3), \
                    offy + self.hexLineSize * math.sin(math.pi * (s + 1) / 3))

            if linex1 < self.minx:
                self.minx = linex1

            if liney1 < self.miny:
                self.miny = liney1

            if linex1 > self.maxx:
                self.maxx = linex1

            if liney1 > self.maxy:
                self.maxy = liney1;

            self.sb += "{},{} ".format(linex1, liney1)
            
        self.sb += "\" style=\"fill: rgb({} {} {}); stroke:black; stroke-width:2\" />\r\n".format(*fillColor)

        lines = message.split(" ")

        for p in range(len(lines)):
            self.sb += "<text text-anchor=\"middle\" dominant-baseline=\"middle\" x=\"{}\" y=\"{}\" fill=\"black\">{}</text>\r\n".format( \
                offx, offy + (p - 0.5 * (len(lines) - 1)) * self.stringHeight, lines[p].replace("&", "&amp;").replace("|", " "))

    def render(self):
        """Returns SVG file"""
        
        self.minx -= 10
        self.miny -= 10
        self.maxx += 10
        self.maxy += 10
        
        viewBox = "{} {} {} {}".format(self.minx, self.miny, self.maxx - self.minx, self.maxy - self.miny)

        return "<svg xmlns=\"http://www.w3.org/2000/svg\" viewBox=\"{}\">\r\n{}</svg>".format(viewBox, self.sb)


In [4]:
def main(categories, hexagons):
    """Main code that generates the SVG file using given CATEGORIES and HEXAGON description"""
    
    colorPalette = PaleColorPalette(len(categories))

    svg = HexSVG()

    for j, row in enumerate(hexagons):
        for i, node in enumerate(row):
            if node is not None:
                topic = node["topic"]
                category = node["category"];
                categoryIndex = categories.index(node["category"])

                fillColor = colorPalette.PALE_COLORS[categoryIndex]

                svg.addHex(i, j, topic, fillColor)
                
    return svg.render()
           
svgContent = main(CATEGORIES, HEXAGONS)

In [5]:
# Displaying resulting SVG content as image in notebook

from IPython.display import SVG

SVG(svgContent)

In [6]:
# Uncomment the following line to make the SVG content visible. You can then copy it, 
# save it as a .svg file, and use it for your personal use.

# print(svgContent)